### For Graduate QM Term Project 2023
Geraldine

# Quantum Cryptography – BB84

For the term project, I'm thinking of exploring the topic of quantum cryotpgraphy algorithms in greater detail, with BB84 as an example (no entanglement). For the computational part, I will probably attempt to simulate the protocol and compare the error rates for the cases in which (1) there is no eavesdropper, and (2) there is an eavesdropper. I will also touch on the E91 protocol, which makes use of entanglement (but will not simulate it).

## Preliminaries – Defining functions used

- `encode(bitlist)`
    - yee.
- `decode(bitlist)`
    - yee.
- `match(ebase, dbase)`
    - yee.
- `compare(ematch, dmatch, d)`
    - yee.

In [1]:
import numpy as np
import sympy as sp
import random

In [2]:
def encode(bitlist):
    
    encoded = []
    ebase = []

    for bit in bitlist:
        
        base = int(np.random.choice([0,1]))
        ebase.append(base)

        if base == 0:   # computational basis +
            if bit == 0:   # vertical
                ebit = "v"
            else:          # horizontal
                ebit = "h"

        else:           # hadamard basis X
            if bit == 0:   # diagonal left
                ebit = "dl"
            else:          # diagonal right
                ebit = "dr"
        
        encoded.append(ebit)
        
    return encoded, ebase

In [3]:
def decode(bitlist):
    
    decoded = []
    dbase = []
    
    for bit in bitlist:
    
        base = int(np.random.choice([0,1]))
        dbase.append(base)
    
        if base == 0:   # computational basis +
            if bit == "v" or bit == "h":   # encode basis = decode basis
                dbit = bit
            else:                          # encode basis != decode basis
                
                # randomly decide on v or h in computational basis
                randnum = int(np.random.choice([0,1]))
                if randnum == 0:
                    dbit = "v"
                else:
                    dbit = "h"

        else:           # hadamard basis X
            if bit == "dl" or bit == "dr":   # encode basis = decode basis
                dbit = bit
            else:                            # encode basis != decode basis
                
                # randomly decide on dl or dr in hadamard basis
                randnum = int(np.random.choice([0,1]))
                if randnum == 0:
                    dbit = "dl"
                else:
                    dbit = "dr"
    
        decoded.append(dbit)
        
    return decoded, dbase

In [4]:
def match(ebase, dbase):
    
    matchindex = []
    matchcount = 0
    
    for i in range(len(ebase)):
        if ebase[i] == dbase[i]:
            matchindex.append(i)
            matchcount += 1
    
    ematch = []
    dmatch = []  
    
    for index in matchindex:
        ematch.append(encoded[index])
        dmatch.append(decoded[index])
    
    return matchindex, matchcount, ematch, dmatch

In [14]:
def compare(ematch, dmatch, d):
    
    efinal = ematch
    dfinal = dmatch
    
    tot_index = list(range(len(ematch)))
    random.shuffle(tot_index)
    
    subsetindex = tot_index[:d]
    subsetindex.sort()
    subsetindex.reverse()
    
    #print(f"indexes of subset: {subsetindex}")
    
    esub = []
    dsub = []
    count = 0
    
    for i in subsetindex:
        esub.append(ematch.pop(i))
        dsub.append(dmatch.pop(i))
    
    #print(f"subset from alice: {esub}")
    #print(f"subset from bob: {dsub}\n")
    
    for j in range(len(subsetindex)):
        if esub[j] != dsub[j]:
                count += 1
    
    error = count/len(subsetindex)
    
    detected = "no eavesdropper detected!"
    
    if abs((0.25 - error)/0.25) < 0.9:
        detected = "eavesdropper detected!"
    
    return esub, dsub, efinal, dfinal, error, detected

## 1. Error Rate (Without Eavesdropper)

yee.

### Mini test run

In [20]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 8
d = 0
numbits = (4+d)*n

print("\n----- SIMULATION START (WITHOUT EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

print(f"encoded bits: {encoded}")

print("\nBOB DECODING BITS IN RANDOM BASIS......")

decoded, dbase = decode(encoded)

print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
print(f"encoding basis: {ebase}")
print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
print(f"indexes where basis match: {matchindex}")
print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
print(f"checked bits (encoded, {len(esub)} bits) = {esub}")
print(f"checked bits (decoded, {len(dsub)} bits) = {dsub}\n")

print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITHOUT EAVESDROPPER) -----

ALICE GENERATING 32 BITS......
alice bits: [0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 0]

ALICE ENCODING BITS IN RANDOM BASIS......
encoded bits: ['dl', 'dr', 'h', 'dl', 'dr', 'dl', 'v', 'h', 'dr', 'h', 'v', 'dl', 'dl', 'dr', 'dl', 'h', 'v', 'dl', 'h', 'dr', 'h', 'dr', 'v', 'h', 'v', 'dl', 'dr', 'dl', 'v', 'dr', 'dr', 'dl']

BOB DECODING BITS IN RANDOM BASIS......
decoded bits: ['dl', 'h', 'dr', 'dl', 'dr', 'h', 'v', 'h', 'dr', 'h', 'v', 'v', 'h', 'h', 'dl', 'dl', 'dr', 'v', 'dl', 'h', 'dl', 'h', 'dl', 'dl', 'dr', 'dl', 'h', 'dl', 'dl', 'h', 'v', 'dl']

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......
encoding basis: [1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1]
decoding basis: [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1]

ALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......
basis matched 12/32 tim

### Full simulation

In [22]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 1000
d = 6
numbits = (4+d)*n

print("\n----- SIMULATION START (WITHOUT EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
#print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

#print(f"encoded bits: {encoded}")

print("\nBOB DECODING BITS IN RANDOM BASIS......")

decoded, dbase = decode(encoded)

#print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
#print(f"encoding basis: {ebase}")
#print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
#print(f"indexes where basis match: {matchindex}")
#print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
#print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
#print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
#print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITHOUT EAVESDROPPER) -----

ALICE GENERATING 10000 BITS......

ALICE ENCODING BITS IN RANDOM BASIS......

BOB DECODING BITS IN RANDOM BASIS......

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......

ALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......
basis matched 4942/10000 times.

ALICE AND BOB COMPARE A SUBSET OF 1000 BITS TO CHECK ERROR RATE......
error rate = 0.0
no eavesdropper detected!


## 2. Error Rate (With Eavesdropper)

yee.

### Mini test run

In [18]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 8
d = 0
numbits = (4+d)*n

print("\n----- SIMULATION START (WITH EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

print(f"encoded bits: {encoded}")

print("\nEVE DECODING BITS IN RANDOM BASIS......")

decoded_eve, dbase_eve = decode(encoded)
print(f"decoded bits BY EVE: {decoded_eve}")

print("\nBOB MEASURING BITS FROM EVE IN RANDOM BASIS......")

decoded, dbase = decode(decoded_eve)

print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
print(f"encoding basis: {ebase}")
print(f"eve's dc basis: {dbase_eve}")
print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
print(f"indexes where basis match: {matchindex}")
print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
print(f"checked bits (encoded, {len(esub)} bits) = {esub}")
print(f"checked bits (decoded, {len(dsub)} bits) = {dsub}\n")

print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITH EAVESDROPPER) -----

ALICE GENERATING 32 BITS......
alice bits: [0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 0 1 0 0 1]

ALICE ENCODING BITS IN RANDOM BASIS......
encoded bits: ['dl', 'h', 'v', 'dl', 'dl', 'dl', 'h', 'v', 'dr', 'v', 'v', 'dl', 'dl', 'v', 'h', 'v', 'h', 'dr', 'dr', 'h', 'v', 'h', 'dl', 'dr', 'v', 'v', 'dr', 'v', 'h', 'dl', 'dl', 'dr']

EVE DECODING BITS IN RANDOM BASIS......
decoded bits BY EVE: ['v', 'dr', 'dl', 'h', 'v', 'dl', 'h', 'v', 'dr', 'dl', 'v', 'h', 'dl', 'dl', 'dr', 'dl', 'dl', 'dr', 'dr', 'dl', 'dr', 'h', 'h', 'h', 'dl', 'dr', 'dr', 'v', 'dr', 'h', 'dl', 'dr']

BOB MEASURING BITS FROM EVE IN RANDOM BASIS......
decoded bits: ['dl', 'v', 'v', 'h', 'dl', 'h', 'h', 'v', 'dr', 'v', 'dr', 'dl', 'dl', 'h', 'dr', 'dl', 'v', 'dr', 'dr', 'dl', 'v', 'h', 'dr', 'h', 'dl', 'h', 'dr', 'v', 'h', 'dr', 'h', 'dr']

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......
encoding basis: [1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,

### Full simulation

In [23]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 1000
d = 6
numbits = (4+d)*n

print("\n----- SIMULATION START (WITH EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
#print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

#print(f"encoded bits: {encoded}")

print("\nEVE DECODING BITS IN RANDOM BASIS......")

decoded_eve, dbase_eve = decode(encoded)
#print(f"decoded bits BY EVE: {decoded_eve}")

print("\nBOB MEASURING BITS FROM EVE IN RANDOM BASIS......")

decoded, dbase = decode(decoded_eve)

#print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
#print(f"encoding basis: {ebase}")
#print(f"eve's dc basis: {dbase_eve}")
#print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
#print(f"indexes where basis match: {matchindex}")
#print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
#print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
#print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
#print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITH EAVESDROPPER) -----

ALICE GENERATING 10000 BITS......

ALICE ENCODING BITS IN RANDOM BASIS......

EVE DECODING BITS IN RANDOM BASIS......

BOB MEASURING BITS FROM EVE IN RANDOM BASIS......

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......

ALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......
basis matched 5103/10000 times.

ALICE AND BOB COMPARE A SUBSET OF 1000 BITS TO CHECK ERROR RATE......
error rate = 0.257
eavesdropper detected!


## Conclusion

yee